In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.dropout import Dropout

import time


from PIL import Image
from tqdm.auto import tqdm

import random
import numpy as np
import os
import cv2

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import timm

import pandas as pd
import csv

import wandb

In [2]:
CFG = {
    'IMG_WIDTH':720,
    'IMG_HEIGTH':1280,
    'EPOCHS':3,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':8,
    'SEED':3
}

In [3]:
# wandb.init(
#     project="Final Project", 
#     entity="aitech4_cv3",
#     name='classification_resnet50',
#     config = {
#         "lr" : CFG['LEARNING_RATE'],
#         "epoch" : CFG['EPOCHS'],
#         "batch_size" : CFG['BATCH_SIZE'],
#     }
#     )

# config = wandb.config

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
data_file = pd.read_csv('/opt/ml/data_csv/data.csv')
label = data_file['info']
data =  data_file.drop(['info',"Unnamed: 0"], axis=1)
X_train, y_train, X_test, y_test = train_test_split(data, label, test_size=0.2, random_state=CFG['SEED'])


In [7]:
train_transform = A.Compose([A.Resize(CFG['IMG_HEIGTH'],CFG['IMG_WIDTH']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])


val_transform = A.Compose([
                            A.Resize(CFG['IMG_HEIGTH'],CFG['IMG_WIDTH']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])



test_transform = A.Compose([
                            A.Resize(CFG['IMG_HEIGTH'],CFG['IMG_WIDTH']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [8]:
X_train = X_train.reset_index()
X_test = X_test.reset_index()
y_train = y_train.reset_index()
y_test = y_test.reset_index()

In [9]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths['filepath']
        self.labels = labels['info']
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_paths[index]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
            
        
        if self.labels is not None:
            label = self.labels[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_paths)

In [10]:
train_dataset = CustomDataset(X_train, X_test, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(y_train, y_test, val_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [11]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=1):
        super(BaseModel, self).__init__()
        self.backbone = timm.create_model(model_name='efficientnet_b0', pretrained=True)
      
        self.conv_stem = self.backbone.conv_stem
        self.bn1 = self.backbone.bn1
        self.act1 = self.backbone.act1
        self.init_block =  nn.Sequential(self.conv_stem, self.bn1, self.act1)

        self.blocks = self.backbone.blocks[0:7]

        self.conv_head = self.backbone.conv_head
        self.bn2 = self.backbone.bn2
        self.act2 = self.backbone.act2
        self.global_pool = self.backbone.global_pool

        self.classifier = nn.Linear(1280,num_classes)
        
    def forward(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)

        x = self.blocks(x)

        x = self.conv_head(x)
        x = self.bn2(x)
        x = self.act2(x)
        cam_feature = nn.ReLU()(x) 

        x = self.global_pool(x)
        x = self.classifier(x)
        x = nn.Sigmoid()(x)
        return x, cam_feature


In [12]:
def calculate_accuracy(y_pred, y):
    top_pred = torch.round(y_pred)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [13]:
model = timm.create_model(model_name='efficientnet_b0', pretrained=True)
print(model)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
   

In [14]:
def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    val_acc = []
    
    with torch.no_grad():
        for img, label in iter(test_loader):
            img, label = img.float().to(device), label.float().to(device).reshape(-1,1)
            
            model_pred,_ = model(img)

            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())

            val_acc.append(calculate_accuracy(model_pred, label).item())
            
            true_labels += label.detach().cpu().numpy().tolist()

    return np.mean(val_loss), np.mean(val_acc)

In [15]:
def time_of_epoch(start, end):
    time_per_epoch = end - start
    time_per_epoch_min = int(time_per_epoch / 60)
    time_per_epoch_sec = int(time_per_epoch -(time_per_epoch_min*60))
    return time_per_epoch_min, time_per_epoch_sec

In [16]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = nn.BCELoss().to(device)

    best_score = 0
    best_model = None

    for epoch in range(1, CFG["EPOCHS"]+1):
        model.train()
        start_time = time.monotonic()
        train_loss = []
        epoch_acc = 0
        for step,(img, label) in enumerate(train_loader): #tqdm(iter(train_loader)

            img, label = img.float().to(device), label.float().to(device)
            optimizer.zero_grad()
            label = label.view(-1,1)

            model_pred,_ = model(img)

            acc = calculate_accuracy(model_pred, label)

            loss = criterion(model_pred, label)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

            if (step + 1) % 5 == 0:
                print(f'Epoch [{epoch}], Step [{step+1}], Train Loss : [{round(loss.item(),4):.5f}] Train acc : [{acc:.5f}]')

        end_time = time.monotonic()
        epoch_min, epoch_sec = time_of_epoch(start_time, end_time)
        train_loss_m = np.mean(train_loss)
        epoch_acc += acc.item()
        val_loss, val_acc = validation(model, criterion, test_loader, device)

        print(f'Epoch [{epoch}], Train Loss : [{train_loss_m:.5f}] Val Loss : [{val_loss:.5f}] Val acc : [{val_acc:.5f}],Time : {epoch_min}m {epoch_sec}s')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_score < val_acc:
            best_model = model
            best_score = val_acc
            print(f"save_best_pth EPOCH {epoch}")
            torch.save(model.state_dict(),"../../model_save_dir/best_cam.pth")
        
    return best_model


In [17]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None
infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Epoch [1], Step [5], Train Loss : [0.67370] Train acc : [0.75000]
Epoch [1], Step [10], Train Loss : [0.48200] Train acc : [0.87500]
Epoch [1], Step [15], Train Loss : [0.42420] Train acc : [0.87500]
Epoch [1], Step [20], Train Loss : [0.53040] Train acc : [0.62500]
Epoch [1], Step [25], Train Loss : [0.56030] Train acc : [0.62500]
Epoch [1], Step [30], Train Loss : [0.30840] Train acc : [1.00000]
Epoch [1], Step [35], Train Loss : [0.45700] Train acc : [0.62500]
Epoch [1], Step [40], Train Loss : [0.66150] Train acc : [0.62500]
Epoch [1], Step [45], Train Loss : [0.63550] Train acc : [0.75000]
Epoch [1], Step [50], Train Loss : [0.26390] Train acc : [1.00000]
Epoch [1], Step [55], Train Loss : [0.45530] Train acc : [0.75000]
Epoch [1], Step [60], Train Loss : [0.38190] Train acc : [0.87500]
Epoch [1], Step [65], Train Loss : [0.22120] Train acc : [1.00000]
Epoch [1], Step [70], Train Loss : [0.51740] Train acc : [0.75000]
Epoch [1], Step [75], Train Loss : [0.79730] Train acc : [0.625

In [35]:
test_img_paths = "/opt/ml/img_data/pothole/V0F_HY_0377_20210107_154114_N_CH0_Busan_Sun_Highway_Day_59153.png"

In [36]:
class CustomTestDataset(Dataset):
    def __init__(self, img_paths, labels, transforms=None):
        self.img_paths = img_paths
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_paths

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        return image
    
    def __len__(self):
        return len(self.img_paths)

In [37]:
test_dataset = CustomTestDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [38]:
def inference(model, img_path, device):
    model = BaseModel()
    model.load_state_dict(torch.load("/opt/ml/model_save_dir/best_cam.pth", map_location=device))
    model = model.to(device)
    
    model_preds = []
    img_paths = []
    img_paths.append(img_path)
    
    model.eval()
    with torch.no_grad():
        for img in iter(test_loader):
            img = img.float().to(device)
            model_pred, c_map = model(img)
            model_preds.append(torch.round(model_pred).detach().cpu().numpy())
    c_map = nn.ReLU()(c_map)
    c_map_w = nn.AdaptiveAvgPool2d(output_size=(1,1))(c_map)
    print(c_map_w.shape)
    print('Done.')
    return model_preds[0][0], c_map, c_map_w

In [39]:
def imwrite(img):
    img = img / 2 + 0.5     # unnormalize
    cv2.imwrite("/opt/ml/cam_dir/c_0.png",np.transpose(img, (1, 2, 0)))

In [40]:
preds, c_map, c_map_w = inference(infer_model, test_img_paths, device)
c_map_np = c_map.detach().cpu().numpy()
c_map_w = c_map_w.detach().cpu().numpy()

print(c_map_np[0])
print(c_map_w[0].shape)
# print(c_map_np[0])
# print(c_map_w[0])

cam_result = c_map_np[0]*c_map_w[0]
cam_result = torch.tensor(cam_result)

cam_result = torch.sum(cam_result, axis=0)
cam_result = cam_result.detach().cpu().numpy()
c_feature_resize = cv2.resize(cam_result,(1280,720),interpolation=cv2.INTER_LINEAR)
cv2.imwrite("/opt/ml/cam_dir/c_0.png",c_feature_resize)

torch.Size([2, 1280, 1, 1])
Done.
[[[0.0000000e+00 0.0000000e+00 2.4601144e-01 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [1.8373414e+00 1.4023100e+00 9.0251476e-01 ... 2.6767367e-02
   0.0000000e+00 0.0000000e+00]
  [3.5191555e+00 2.3378236e+00 1.1276944e+00 ... 5.4869664e-01
   1.4029621e+00 0.0000000e+00]
  ...
  [0.0000000e+00 6.1183429e-01 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 7.2381264e-01 2.1132527e-01 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]]

 [[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
   0.0000000e+00 0.0000000e+00]
  

True